### На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue. Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.



In [39]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

# Поработаем с датасетом c покупками

# Подготовка данных

In [40]:
data = pd.read_csv('gb_sem_8_hm.csv')

data.head(10)

,USER_ID;VARIANT_NAME; REVENUE
737;variant;0,0
2423;control;0,0
9411;control;0,0
7311;control;0,0
6174;variant;0,0
2380;variant;0,0
2849;control;0,0
9168;control;0,0
6205;variant;0,0
7548;control;0,0


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 737;variant;0 to 9468;control;0
Data columns (total 1 columns):
 #   Column                          Non-Null Count  Dtype
---  ------                          --------------  -----
 0   USER_ID;VARIANT_NAME; REVENUE   10000 non-null  int64
dtypes: int64(1)
memory usage: 156.2+ KB


In [42]:
data.shape

(10000, 1)

In [43]:
data.describe()

,USER_ID;VARIANT_NAME; REVENUE
count,10000.000000
mean,0.644700
std,6.528501
min,0.000000
25%,0.000000
50%,0.000000
75%,0.000000
max,99.000000


Посмотрим на группы отдельно

In [44]:
control = data[data['ab_group'] == 'A'].copy(deep=True)
treatment = data[data['ab_group'] == 'B'].copy(deep=True)

KeyError: 'ab_group'

In [ ]:
control.describe()

,revenue,num_purchases,purchase,av_site visit
count,11835.000000,11835.000000,11835.000000,11835.000000
mean,0.404462,0.050697,0.021631,6.974724
std,13.133218,1.467511,0.145481,2.023533
min,0.000000,0.000000,0.000000,-12.073486
25%,0.000000,0.000000,0.000000,5.656155
50%,0.000000,0.000000,0.000000,6.982329
75%,0.000000,0.000000,0.000000,8.345572
max,1303.609284,152.000000,1.000000,17.728836


In [ ]:
treatment.describe()

,revenue,num_purchases,purchase,av_site visit
count,11817.000000,11817.000000,11817.000000,11817.000000
mean,0.244794,0.036473,0.019802,7.051559
std,3.176534,0.418480,0.139325,3.976799
min,0.000000,0.000000,0.000000,-8.286822
25%,0.000000,0.000000,0.000000,4.380984
50%,0.000000,0.000000,0.000000,7.060873
75%,0.000000,0.000000,0.000000,9.768648
max,113.830000,25.000000,1.000000,22.446822


Пока мы еще ничего не посчитали, но уже можно заметить, что максимальный чек в первой группе сильно больше, чем в группе B. Все мы знаем, что среднее очень неустойчиво к выбросам, так что нам необходимо будет это учесть.

# Применение статистических критериев

In [ ]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

In [ ]:
def proportion_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий
    size = control.loc[:, column].shape[0]
    prop_control = control.loc[:, column].sum() / size
    prop_treatment = treatment.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = stats.bernoulli.rvs(p=prop_control, size=size)
        treatment_sample = stats.bernoulli.rvs(p=prop_treatment, size=size)
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_control, prop_treatment)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column]) 
    return result

## Метрика визиты на юзера

In [ ]:
import plotly.express as px
fig = px.histogram(data,
                   x='av_site visit',
                   color = 'ab_group',
                   title='avg_site_visits_distribution',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()

ModuleNotFoundError: No module named 'plotly'

In [ ]:
continious_result(control, treatment, column='av_site visit')

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:06<00:00, 1545.12it/s]


,effect_size,alpha,beta,CI,difference
av_site visit,0.024352,0.061387,0.534515,"[-0.156, 0.004]",-0.076748


## Метрика визиты на юзера с покупкой

In [ ]:
fig = px.histogram(data[data['purchase'] == 1],
                   x='av_site visit',
                   color = 'ab_group',
                   title='avg_site_visits_distribution',
                   marginal = 'box',
                   nbins = 50,
                   barmode='overlay')
fig.show()

In [ ]:
continious_result(control[control.purchase == 1], 
                  treatment[treatment.purchase == 1],
                  column='av_site visit')

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:02<00:00, 4657.66it/s]


,effect_size,alpha,beta,CI,difference
av_site visit,0.029001,0.731283,0.937626,"[-0.648, 0.456]",-0.096075


## Метрика конверсия в покупку

In [ ]:
fig = px.histogram(data, x="purchase",
                   color='ab_group', barmode='group',
                   height=400)
fig.show()

In [ ]:
proportion_result(control, treatment, column='purchase')

100%|██████████████████████████████████████████████████████████████████████████| 10000/10000 [00:04<00:00, 2263.98it/s]


,effect_size,alpha,beta,CI,difference
purchase,0.013059,0.317813,0.828798,"[-0.002, 0.005]",0.001859


## Рассмотрим кейс с рекламной выручкой

In [ ]:
df = pd.read_excel('data/gb_sem_8_hm.xlsx')

D:\software\anaconda\lib\site-packages\openpyxl\worksheet\_read_only.py:79: UserWarning:

Unknown extension is not supported and will be removed



In [ ]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [ ]:
df.shape

(10000, 3)

In [ ]:
df.USER_ID.nunique()

6324

In [ ]:
df = df.groupby(['USER_ID', 'VARIANT_NAME'], as_index=False).agg({'REVENUE': 'sum'})

In [ ]:
df.shape

(7865, 3)

In [ ]:
df.groupby('USER_ID', as_index=False).agg({'VARIANT_NAME': 'count'})['VARIANT_NAME'].value_counts()

1    4783
2    1541
Name: VARIANT_NAME, dtype: int64

In [ ]:
unique_ids = \
(df
 .groupby('USER_ID', as_index=False)
 .agg({'VARIANT_NAME': 'count'})
 #.['VARIANT_NAME'].value_counts()
 .query('VARIANT_NAME == 1')
 .USER_ID
 .values
 )

In [ ]:
df_new = df[df.USER_ID.isin(unique_ids)].copy(deep=True)

In [ ]:
df_new.describe()

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000
